In [1]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np

from Constants import Constants as const

In [10]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20240825_stock_act_reg_data.dta'))
new_year_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'new_shock_year_data_compustat.csv'),
                                     usecols=[const.GVKEY, 'year', 'post', 'freq']).rename(
    columns={'post': 'post60min', 'freq': 'freq60min', 'year': 'fiscal_year'})
# new_quarterly_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'new_year_quarter_data.csv'))

In [14]:
reg_df_60: DataFrame = reg_df.merge(new_year_df, on=[const.GVKEY, const.YEAR], how='left').drop(['busdesc'], axis=1)
event_index = reg_df_60.loc[reg_df_60[const.YEAR].apply(lambda x: 2008 < x < 2015)].index
reg_df_60.loc[event_index, 'freq60min'] = reg_df_60['freq60min'].fillna(0)
reg_df_60.loc[event_index, 'ln_freq60min'] = reg_df_60['freq60min'].apply(lambda x: np.log(x + 1))
reg_df_60.loc[event_index, 'freq60min'] = (reg_df_60.loc[event_index, const.YEAR] > 2011).astype(int)
reg_df_60.to_stata(os.path.join(const.RESULT_PATH, '20240918_stock_act_reg_data.dta'),
                   write_index=False)

In [20]:
# merge quarterly data
new_quarterly_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'new_year_quarter_data_gvkey.csv'))
majgovcustomer_gvkey = reg_df.loc[reg_df['MajorGovCustomer'] == 1, 'gvkey'].unique()
new_quarterly_df.loc[:, 'MajorGovCustomer'] = 0
new_quarterly_df.loc[new_quarterly_df[const.GVKEY].isin(majgovcustomer_gvkey), 'MajorGovCustomer'] = 1

new_quarterly_df['yearquarter'] = new_quarterly_df.apply(lambda x: '{}q{}'.format(int(x['Year']), int(x['Quarter'])), axis=1)
new_quarterly_df.loc[:, 'post'] = (new_quarterly_df['Year'] > 2011).astype(int)
new_quarterly_df['ln_freq'] = new_quarterly_df['freq'].apply(lambda x: np.log(x + 1))

int_terms = list()
for year in range(2008, 2015):
    for quarter in range(1, 5):
        new_quarterly_df.loc[:, f'd{year}q{quarter}'] = (new_quarterly_df['yearquarter'] == f'{year}q{quarter}').astype(int)
        int_terms.append(f'1.d{year}q{quarter}#1.MajorGovCustomer')

new_quarterly_df.to_stata(os.path.join(const.RESULT_PATH, '20240918_stock_act_reg_data_quarterly.dta'), write_index=False)

print(' '.join(int_terms))

1.d2008q1#1.MajorGovCustomer 1.d2008q2#1.MajorGovCustomer 1.d2008q3#1.MajorGovCustomer 1.d2008q4#1.MajorGovCustomer 1.d2009q1#1.MajorGovCustomer 1.d2009q2#1.MajorGovCustomer 1.d2009q3#1.MajorGovCustomer 1.d2009q4#1.MajorGovCustomer 1.d2010q1#1.MajorGovCustomer 1.d2010q2#1.MajorGovCustomer 1.d2010q3#1.MajorGovCustomer 1.d2010q4#1.MajorGovCustomer 1.d2011q1#1.MajorGovCustomer 1.d2011q2#1.MajorGovCustomer 1.d2011q3#1.MajorGovCustomer 1.d2011q4#1.MajorGovCustomer 1.d2012q1#1.MajorGovCustomer 1.d2012q2#1.MajorGovCustomer 1.d2012q3#1.MajorGovCustomer 1.d2012q4#1.MajorGovCustomer 1.d2013q1#1.MajorGovCustomer 1.d2013q2#1.MajorGovCustomer 1.d2013q3#1.MajorGovCustomer 1.d2013q4#1.MajorGovCustomer 1.d2014q1#1.MajorGovCustomer 1.d2014q2#1.MajorGovCustomer 1.d2014q3#1.MajorGovCustomer 1.d2014q4#1.MajorGovCustomer


In [24]:
inter_terms = '1.d2009q1#1.MajorGovCustomer 1.d2009q2#1.MajorGovCustomer 1.d2009q3#1.MajorGovCustomer 1.d2009q4#1.MajorGovCustomer 1.d2010q1#1.MajorGovCustomer 1.d2010q2#1.MajorGovCustomer 1.d2010q3#1.MajorGovCustomer 1.d2010q4#1.MajorGovCustomer 1.d2011q1#1.MajorGovCustomer 1.d2011q2#1.MajorGovCustomer 1.d2011q3#1.MajorGovCustomer 1.d2011q4#1.MajorGovCustomer 1.d2012q1#1.MajorGovCustomer 1.d2012q2#1.MajorGovCustomer 1.d2012q3#1.MajorGovCustomer 1.d2012q4#1.MajorGovCustomer 1.d2013q1#1.MajorGovCustomer 1.d2013q2#1.MajorGovCustomer 1.d2013q3#1.MajorGovCustomer 1.d2013q4#1.MajorGovCustomer 1.d2014q1#1.MajorGovCustomer 1.d2014q2#1.MajorGovCustomer 1.d2014q3#1.MajorGovCustomer 1.d2014q4#1.MajorGovCustomer'.split(' ')
xlabel = list()
for i, term in enumerate(inter_terms):
    year = term.split('q')[0][-4:]
    quarter = term.split('q')[1][0]
    xlabel.append(f'{i+1} "{year}q{quarter}"')
    
print(' '.join(xlabel))

1 "2009q1" 2 "2009q2" 3 "2009q3" 4 "2009q4" 5 "2010q1" 6 "2010q2" 7 "2010q3" 8 "2010q4" 9 "2011q1" 10 "2011q2" 11 "2011q3" 12 "2011q4" 13 "2012q1" 14 "2012q2" 15 "2012q3" 16 "2012q4" 17 "2013q1" 18 "2013q2" 19 "2013q3" 20 "2013q4" 21 "2014q1" 22 "2014q2" 23 "2014q3" 24 "2014q4"


# Merge bargaining ratio data